- 参照
    - [RAGコンペ参加記 (raggle)](https://qiita.com/ctc-j-ikai/items/9980f6a1c11ef444ba4d)

# Import

In [1]:
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "rag-rohto"

# Document Loader

In [2]:
from langchain_community.document_loaders import PyPDFLoader
import pdfplumber


def load_pdf_document(file_path):
    documents = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            documents.append({"content": page.extract_text(), "metadata": {"source": file_path}})
    return documents


file_paths = [
    "../dataset/Financial_Statements_2023.pdf",
    "../dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf",
    "../dataset/Shibata_et_al_Research_Article.pdf",
    "../dataset/V_Rohto_Premium_Product_Information.pdf",
    "../dataset/Well-Being_Report_2024.pdf",
]

# PDFファイルを読み込む
docs = [load_pdf_document(file_path) for file_path in file_paths]
# リストをフラット化
docs_list = [item for sublist in docs for item in sublist]

# Document transformer

In [3]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# `docs_list` を Document 型に変換
docs_list_converted = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in docs_list]

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
)

doc_splits = text_splitter.split_documents(docs_list_converted)
print(len(doc_splits))

Created a chunk of size 1097, which is longer than the specified 1000


514


# Embedding model

In [4]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vector store

In [5]:
# from langchain_chroma import Chroma

# db = Chroma.from_documents(doc_splits, embeddings)

# Retriever

In [6]:
# retriever = db.as_retriever()

In [7]:
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from sudachipy import dictionary
from sudachipy import tokenizer
from typing import List, Dict


# 単語単位のn-gramを作成
def generate_word_ngrams(text, i, j, binary=False):
    tokenizer_obj = dictionary.Dictionary(dict="core").create()
    mode = tokenizer.Tokenizer.SplitMode.A
    tokens = tokenizer_obj.tokenize(text, mode)
    words = [token.surface() for token in tokens]

    ngrams = []

    for n in range(i, j + 1):
        for k in range(len(words) - n + 1):
            ngram = tuple(words[k : k + n])
            ngrams.append(ngram)

    if binary:
        ngrams = list(set(ngrams))  # 重複を削除

    return ngrams


def preprocess_word_func(text: str) -> List[str]:
    return generate_word_ngrams(text, 1, 1, True)


# 文字単位のn-gramを作成
def generate_character_ngrams(text, i, j, binary=False):
    ngrams = []

    for n in range(i, j + 1):
        for k in range(len(text) - n + 1):
            ngram = text[k : k + n]
            ngrams.append(ngram)

    if binary:
        ngrams = list(set(ngrams))  # 重複を削除

    return ngrams


def preprocess_char_func(text: str) -> List[str]:
    i, j = 1, 3
    if len(text) < i:
        return [text]
    return generate_character_ngrams(text, i, j, True)

In [8]:
# 単語と文字のBM25Retrieverを作成
word_retriever = BM25Retriever.from_documents(doc_splits, preprocess_func=preprocess_word_func)
char_retriever = BM25Retriever.from_documents(doc_splits, preprocess_func=preprocess_char_func)
word_retriever.k = 4
char_retriever.k = 4

# EnsembleRetrieverを作成
ensemble_retriever = EnsembleRetriever(retrievers=[word_retriever, char_retriever], weights=[0.7, 0.3])

In [9]:
query = "肌ラボ 極潤ヒアルロン液の使用上の注意点を教えてください。"

context_docs = ensemble_retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'source': '../dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf'}
本製品の容器には、環境に配慮したバイオマス原料を⼀部使⽤しています。
＊：加⽔分解ヒアルロン酸（ナノ化ヒアルロン酸）、アセチルヒアルロン酸Na（スーパーヒアルロン酸）、乳酸球菌／ヒアルロン酸発
酵液（乳酸発酵ヒアルロン酸）、ヒアルロン酸Na
◆本品は、航空法で定める航空危険物に該当しません。
★販売名：ハダラボモイスト化粧⽔d
使⽤上の注意
＜相談すること＞
○肌に異常が⽣じていないかよく注意して使⽤すること。使⽤中、⼜は使⽤後⽇光にあたって、⾚み、はれ、か
ゆみ、刺激、⾊抜け（⽩斑等）や⿊ずみ等の異常が現れた時は、使⽤を中⽌し、⽪フ科専⾨医等へ相談するこ
と。そのまま使⽤を続けると症状が悪化することがある。
＜その他使⽤上の注意＞
○傷、はれもの、湿疹等、異常のある部位には使⽤しないこと。
○⽬に⼊らないように注意し、⼊った時はすぐに⽔⼜はぬるま湯で洗い流すこと。なお、異常が残る場合は、眼
科医に相談すること。
肌ラボ 極潤ヒアルロン液に関連する製品
当社は、お客様のウェブ体験の向上のため、アクセスを分析しコンテンツや広告をパーソナライズするためにクッキーを使⽤し Cookie 設定
ます。詳細はプライバシーポリシーをご確認ください。プライバシーポリシー
すべての Cookie を受け⼊れる
https://jp.rohto.com/hadalabo/gokujun-lotion/ 1/2


# LCELを使ったRAGのChainの実装

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    '''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
'''
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = {"context": ensemble_retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

query = "肌ラボ 極潤ヒアルロン液の使用上の注意点を教えてください。"

output = chain.invoke(query)
print(output)

肌ラボ 極潤ヒアルロン液の使用上の注意点は以下の通りです：

1. **肌の異常に注意**: 使用中または使用後に日光にあたって、赤み、はれ、かゆみ、刺激、色抜け（白斑等）や黒ずみ等の異常が現れた場合は、使用を中止し、皮膚科専門医等に相談すること。

2. **異常のある部位には使用しない**: 傷、はれもの、湿疹等、異常のある部位には使用しないこと。

3. **目に入らないように注意**: 目に入った場合はすぐに水またはぬるま湯で洗い流し、異常が残る場合は眼科医に相談すること。

これらの注意点を守って使用することが推奨されています。
